In [1]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry
import json

In [2]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [4]:
region = ee.Geometry.Polygon(
    [
        [
            [-92.67583,42.28862],
            [-92.86461,42.43156],
            [-93.02193,42.28992],
            [-92.86345,42.15832],
        ]
    ],
    None,
    False,
)

# RGB Downloading

In [11]:
Map = geemap.Map()


# Parameters
params = {
    'dimensions': '1000x1000',
    'format': "GEO_TIFF",
    'prefix': 'tile_',
    'processes': 25,
    'out_dir': './All_Data/rgb2so_dataset/1/RGB/2122/',
}

image = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(region)
    .filterDate('2021', '2022')
    .mosaic()
    .clip(region)
    .select('R', 'G', 'B')
)

# Additional Function to create grid cells covering the region
def create_grid(region, dx, dy):
    bounds = region.bounds().getInfo()['coordinates'][0]
    minx = min([coord[0] for coord in bounds])
    miny = min([coord[1] for coord in bounds])
    maxx = max([coord[0] for coord in bounds])
    maxy = max([coord[1] for coord in bounds])
    
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            cell = ee.Geometry.Rectangle([x, y, x + dx, y + dy])
            grid.append(cell)
            y += dy
        x += dx
    return grid

# Define your grid size
dx = 0.02  # Adjust grid size in x
dy = 0.02  # Adjust grid size in y

# Create grid
grid = create_grid(region, dx, dy)

def download_tile(index, cell):
    region = cell.bounds().getInfo()['coordinates']
    url = image.getDownloadURL(
        {
            'region': region,
            'dimensions': params['dimensions'],
            'format': params['format'],
        }
    )
    
    ext = 'tif'
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(len(grid))))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

# Run the Download in Parallel
pool = multiprocessing.Pool(params['processes'])
pool.starmap(download_tile, enumerate(grid))
pool.close()

Done:  012
Done:  000
Done:  054
Done:  027
Done:  042
Done:  039
Done:  003
Done:  009
Done:  015
Done:  051
Done:  048
Done:  060
Done:  030
Done:  033
Done:  021
Done:  072
Done: Done:   045
066
Done:  013
Done:  036
Done:  Done:  024
018
Done:  063Done:  
055Done: 
 057
Done:  028
Done:  006
Done:  040
Done:  069
Done:  001
Done:  043
Done:  025
Done:  004
Done:  052
Done:  014
Done:  016
Done:  034
Done:  067
Done:  041
Done:  022
Done:  049
Done:  026
Done:  029
Done: Done:  073
 010
Done:  061
Done:  037
Done:  002
Done:  031
Done:  053
Done: Done:   019058

Done: Done:   064046

Done:  007
Done:  056
Done:  070
Done:  044
Done:  005
Done:  011
Done:  017
Done:  075
Done:  035
Done:  084
Done:  081
Done:  038
Done:  078
Done:  068
Done:  032
Done:  023Done: 
 074
Done:  090
Done:  020
Done:  062
Done:  050
Done:  071
Done:  087Done:  047

Done:  008
Done:  065
Done:  093
Done:  096
Done:  059
Done:  076
Done:  102
Done:  099
Done:  082
Done:  079
Done:  108
Done:  085
Done:  105

# DEM Downloading

In [12]:
Map = geemap.Map()


# Parameters
params = {
    'dimensions': '2000x2000',
    'format': "GEO_TIFF",
    'prefix': 'tile1_',
    'processes': 25,
    'out_dir': './All_Data/rgb2so_dataset/1/DEM/'
}

image = (
    ee.ImageCollection("USGS/3DEP/1m")
    .filterBounds(region)
    .filterDate('2000', '2020')
    .mosaic()
    .clip(region)
    .select('elevation')
)

# Additional Function to create grid cells covering the region
def create_grid(region, dx, dy):
    bounds = region.bounds().getInfo()['coordinates'][0]
    minx = min([coord[0] for coord in bounds])
    miny = min([coord[1] for coord in bounds])
    maxx = max([coord[0] for coord in bounds])
    maxy = max([coord[1] for coord in bounds])
    
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            cell = ee.Geometry.Rectangle([x, y, x + dx, y + dy])
            grid.append(cell)
            y += dy
        x += dx
    return grid

# Define your grid size
dx = 0.02  # Adjust grid size in x
dy = 0.02  # Adjust grid size in y

# Create grid
grid = create_grid(region, dx, dy)

def download_tile(index, cell):
    region = cell.bounds().getInfo()['coordinates']
    url = image.getDownloadURL(
        {
            'region': region,
            'dimensions': params['dimensions'],
            'format': params['format'],
        }
    )
    
    ext = 'tif'
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(len(grid))))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

# Run the Download in Parallel
pool = multiprocessing.Pool(params['processes'])
pool.starmap(download_tile, enumerate(grid))
pool.close()

Done:  012
Done:  000
Done:  013
Done:  018
Done:  021
Done:  048
Done:  036
Done:  069
Done:  027
Done:  066
Done:  003
Done:  006
Done: Done:  072
 063
Done:  051
Done:  001
Done:  042
Done:  033
Done:  054
Done:  057
Done:  024
Done:  030
Done:  045
Done:  039
Done:  009
Done:  015
Done:  060
Done:  014
Done:  049
Done:  010
Done:  028
Done:  022
Done:  019
Done:  070
Done:  011
Done:  055
Done:  067
Done:  043
Done:  046
Done:  Done: 002 
034
Done:  025
Done:  007
Done:  058
Done:  073
Done:  052
Done:  016
Done:  031
Done:  004
Done:  061
Done:  040
Done:  075
Done:  037
Done:  064
Done:  050
Done:  029
Done:  044
Done:  023
Done:  078
Done:  047
Done:  056
Done:  026
Done:  081
Done:  068
Done:  035
Done:  008
Done:  074
Done:  032
Done:  053
Done:  059
Done:  017
Done:  041
Done:  062
Done:  071
Done:  065
Done:  084
Done:  038
Done:  076
Done:  005
Done:  079
Done:  087
Done:  090
Done:  096
Done:  093
Done:  114
Done:  082
Done:  020
Done:  102
Done:  108
Done:  099
Done:  129

# CropLand Downloading

In [13]:
Map = geemap.Map()


# Parameters
params = {
    'dimensions': '2000x2000',
    'format': "GEO_TIFF",
    'prefix': 'tile_',
    'processes': 25,
    'out_dir': './All_Data/rgb2so_dataset/1/CropLand/'
}

image = (
    ee.ImageCollection("USDA/NASS/CDL")
    .filterBounds(region)
    .filterDate('2021', '2022')
    .mosaic()
    .clip(region)
    .select('cropland')
)

# Additional Function to create grid cells covering the region
def create_grid(region, dx, dy):
    bounds = region.bounds().getInfo()['coordinates'][0]
    minx = min([coord[0] for coord in bounds])
    miny = min([coord[1] for coord in bounds])
    maxx = max([coord[0] for coord in bounds])
    maxy = max([coord[1] for coord in bounds])
    
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            cell = ee.Geometry.Rectangle([x, y, x + dx, y + dy])
            grid.append(cell)
            y += dy
        x += dx
    return grid

# Define your grid size
dx = 0.02  # Adjust grid size in x
dy = 0.02  # Adjust grid size in y

# Create grid
grid = create_grid(region, dx, dy)

def download_tile(index, cell):
    region = cell.bounds().getInfo()['coordinates']
    url = image.getDownloadURL(
        {
            'region': region,
            'dimensions': params['dimensions'],
            'format': params['format'],
        }
    )
    
    ext = 'tif'
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(len(grid))))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

# Run the Download in Parallel
pool = multiprocessing.Pool(params['processes'])
pool.starmap(download_tile, enumerate(grid))
pool.close()

Done:  033Done: 
 024
Done:  060
Done:  066
Done:  063
Done:  036
Done:  045
Done:  048
Done:  054
Done:  021
Done:  072
Done:  015
Done:  018
Done:  012
Done:  006
Done:  003
Done:  042
Done:  030
Done:  051
Done:  009
Done:  057
Done:  000
Done:  069
Done:  039
Done:  027
Done:  061
Done:  067
Done:  022
Done:  073
Done:  007
Done:  004
Done:  034
Done:  055
Done:  025
Done:  019
Done:  016
Done:  037
Done:  010
Done:  043
Done: 
 064Done:  046
Done:  052
Done:  058
Done:  031
Done:  001
Done:  049Done: 
 013
Done:  070
Done:  040
Done:  028
Done:  062
Done:  026
Done:  023
Done:  005
Done:  068
Done:  044
Done:  Done: 074 011

Done:  035
Done:  056
Done:  017
Done:  032
Done:  053
Done:  038
Done:  059
Done:  008
Done:  065
Done:  002
Done:  029
Done:  071
Done:  014
Done:  047
Done:  050
Done:  041
Done: Done:   084020

Done:  075
Done:  108
Done:  081
Done:  078
Done:  102
Done:  117
Done:  093
Done:  090
Done:  114
Done:  120
Done:  105
Done:  111
Done:  132
Done:  099
Done:  087